In [1]:
import camelot
import pandas as pd

In [2]:
# Read all the tables across all pages

# List of Mining Companies
tables = camelot.read_pdf('SOCIETES_MINIERES.pdf', pages="all")

# Concatenating all the tables with pandas
df = pd.concat([table.df for table in tables], ignore_index=True)
df.columns = df.iloc[0]
df = df.drop(index=0).reset_index(drop=True)
df.drop_duplicates(inplace=True)
df.head()

,Nom entreprise,Substance,Adresse Physique,Téléphone,Email
0,2M TRADING MULTISERVICES SARL,,"80, Cité Isra1, Hann Bel Air, Dakar, Sénégal",77 260 85 48,
1,2SBCI,calcaire,"Villa 477, unité 5,Parcelles Assainies, Keur M...",77 434 83 83,
2,2SH SALY SOFTWARE ET HARDWARE,,"Canal 4, en face Maison de la Douane, DAKAR, S...",77 439 28 61,
3,3S INTERNATIONAL,,"Jardin de Fann, Villa n°1, Fann Résidence, Dak...",,
4,AAURIC HOLDINGS PTE.LTD,,"Km 18, Rte de RUFISQUE, BP 3835, MBAO, Sénégal",,


In [3]:
df.to_csv('SOCIETES_MINIERES.csv', index=False)
print(f'Number of Company: {len(df)}')

Number of Company: 745
